## Making sub-images from FITS files

**Author:** Aram Lee, Hossen Teimoorinia
**Date:** 2025-02-15
**File Name:** ImageCutter.ipynb

### [Description]
This file uses Astropy’s CutOut2D function to process images, whether they contain artificial TNOs or not. It can be used to create a training set with randomized cutout positions and generate sub-images from actual images for applying the trained model.

### [Required Libraries]
- numpy: 1.26.4
- astropy: 6.1.0

### [Workflow]  

Steps 1-3 are for training the model, and steps 4-6 are for using the model to detect TNOs.

|Step|File|Input|Output|Purpose|
|-|-|-|-|-|
|1|ImageCutter.ipynb **(Here)**|.fits (with artificial moving objects), .plantlist (artificial objects info)| .npy|Extract sub-images for training|
|2|Concatenator.ipynb|.npy (sub-images from ImageCutter)|.npy|Prepare dataset for training|
|3|Trainer.ipynb|.npy (dataset from Concatenator), .npy (target information)|.h5 (trained CNN models)|Train the model|
|-|-|-|-|-|
|4|ImageCutter.ipynb **(Here)**|.fits (without artificial moving objects)|.npy|Extract sub-images for detection|
|5|Predictor.ipynb|.npy (sub-images from ImageCutter), .npy (target info), .h5 (model)|.npy|Apply trained model to detect objects|
|6a|Link_sources_to_objects.py|.npy (classification and regression output from Predictor)|.npy|Detect moving objects (linear fitting method)|
|6b|CandidateFinder.ipynb|.npy (classification output from Predictor), .npy (sub-images, target info)|.csv|Detect moving objects (scoring method)|

In [2]:
from glob import glob
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D 
from matplotlib import pyplot as plt
from astropy.visualization import ZScaleInterval, ImageNormalize
from itertools import combinations

In [3]:
# load all FITS files to use, and sort them out with the CCD numbers.

ch = "05"
lst = glob(f'allChips/*{ch}.fits')
lst.sort()
print(len(lst))

44


In [4]:
# Create coordinates that are the centres of sub-images. These coordinates create a grid. 
sx = 2048
sy = 4612
dx = 64
dy = 64
n_border = 16

# Define the limits of centre positions
x_min, x_max = dx//2 + n_border, sx - (dx//2 + n_border)
y_min, y_max = dy//2 + n_border, sy - (dy//2 + n_border)

coor_x = np.arange(start=x_min, stop=x_max, step=63)
coor_y = np.arange(start=y_min, stop=y_max, step=63)

# Combine x, y coordinates into 2D positions
coor_ = np.array(np.meshgrid(coor_x, coor_y, indexing='ij')).T.reshape(-1, 2)
np.random.shuffle(coor_)
len_coor_= len(coor_)

# Create indices for image pairs
pairs = list(combinations(range(len(lst)), 2))
pairs = np.array(pairs)
np.random.shuffle(pairs)

In [5]:
# Randomized coordinates can be used to make the training sets instead.

n_samples = 500
coor_x_rand = np.random.randint(0, x_max, n_samples)
coor_y_rand = np.random.randint(y_min, y_max, n_samples)

# Combine x, y coordinates into 2D positions
rand_coor = np.column_stack((coor_x_rand, coor_y_rand))

In [6]:
# Read the head and translate coor_ into the sky coordinates

hdu0 = fits.open(lst[0])[1]  
wcs0 = WCS(hdu0.header) 
coor_w0 = wcs0.wcs_pix2world(coor_,0)

0 = Id, 
1 = x, 
2 = y, 
3 = rate of motion in */hr, 
4 = angle of motion on the sky,
5 = rate in RA, 
6 = rate in Dec, 
7 = mag, 
8 = flux multiplier to achieve correct apparent magnitude.

In [7]:
# Define a function that makes sub-images, and also extract information of moving objects in the sub-images.

def cut_images(coor_w0,lst,n_lst,dx=64,dy=64,n_border=10,overlap=10,sx=2048,sy=4612):
    tp=[]
    file_ = lst[n_lst]
    fits_ = fits.open(file_)
    hdu= fits_[1]
    data = hdu.data
    header=hdu.header
    wcs = WCS(hdu.header)
    plst_ = file_[:-5]+'.plantListrd'
    plst = np.loadtxt(plst_)
    plst = np.array(plst)
    for ks in range(len(coor_w0)):
        n_object=0
        p1=[-1]*11
        kx,ky=wcs.wcs_world2pix(coor_w0[ks,0], coor_w0[ks,1], 0)
        
        for k1 in range(len(plst)):
            xp=plst[k1,1]#-1 # should be corrected based on Fortan and python :
            yp=plst[k1,2]#-1
            if ((xp>=kx-dx//2) & (xp<=kx+dx//2) & (yp>=ky-dy//2) & (yp<=ky+dy//2)):
                try:
                    coor_MO=Cutout2D(data, [kx,ky], [dx,dy]).to_cutout_position((xp, yp))
                except:
                    coor_MO=[dx//2+(xp-kx),dx//2+(yp-ky)]
                coor_object = [coor_MO[0],coor_MO[1],xp,yp,np.float(kx),np.float(ky)]
                pl = plst[k1]
                n_object +=1
                p1 = plst[k1]
            if n_object == 0:
                coor_object = [-1,-1,-1,-1,np.float(kx),np.float(ky)]
                
        try:
            data_ = np.reshape(Cutout2D(data, [kx,ky], [dx,dy]).data,(dx,dy))
            #data_ = np.reshape(Cutout2D(data, [kx,ky], [dx,dy]).data,-1)
            tp.append([file_[:-5], data_,n_object,coor_object,p1])
        except:
            pass
    return tp

In [8]:
# load all data, headers, and WCS information from FITS files.

fits_data = []
fits_header = []
wcs_list = []
for i in lst:
    fits_ = fits.open(i)
    hdu = fits_[1]
    data = hdu.data
    fits_data.append(data)
    header = hdu.header
    fits_header.append(header)
    wcs = WCS(hdu.header)
    wcs_list.append(wcs)

In [2]:
"""
Make image pairs using the cut_images function.

If used in Step 4, adjust this cell to generate sub-images without artificial moving objects.  
The cut_images function should also be simplified for images without artificial moving objects.  
In that case, no target accompanies the images.
"""

import time
from multiprocessing import Pool
from multiprocessing import Process

n_cuts = len(coor_w0) # the number of cutout images to make
n_pairs = len(pairs)
t0=time.asctime()
print(t0)
def cutimage(k):
    im_pair = []
    im_pair.append(cut_images(coor_w0[0:n_cuts],lst,k[0]))
    im_pair.append(cut_images(coor_w0[0:n_cuts],lst,k[1]))
    if np.shape(im_pair) == (2, len(coor_w0), 5):
        return im_pair
    
# Utilize multiprocessing
with Pool() as p:
    result = p.map(cutimage, pairs)

imt = np.array(result)
t1 = time.asctime()
del result

print(np.shape(imt))
print(t1)

In [39]:
# remove None values from imt (the list of image pairs created)
imt = [e for e in imt if e is not None]
imt = np.array(imt, dtype='object')

In [15]:
# Save the sub-images
np.save('imt', imt)

In [16]:
# # Load the sub-images
# imt = np.load('imt.npy', allow_pickle=True)

In [20]:
# make sub-image pairs from the sub-images

# count the number of sub-images
sz_sam,_,sz_img,_= np.shape(imt)

n_n=0  # number of negative (pair of ) images
n_p=0  # number of positive (pair of ) images
n_pp=0  # number of positive (pair of ) images with moire than one moving object in a 64x64 image

inp_pp=[] # input positive (with more than 1 object in a 64x64 image). For now, we are not using it.
tar_pp=[] # target positive

inp_p=[] # input positive
inp_n=[] # input negative 
tar_p=[] # target (positive)
tar_n=[] # target (negative)

for kz in range(sz_img):
    for k1 in range(sz_sam):
        if ((imt[k1,0,kz,2]==1) & (imt[k1,1,kz,2]==1)): 
            inp_p.append([imt[k1,0,kz,1],imt[k1,1,kz,1]])
            tar_p.append([1,1,imt[k1,0,kz,3],imt[k1,1,kz,3],imt[k1,0,kz,0],imt[k1,1,kz,0],imt[k1,0,kz,4],imt[k1,1,kz,4]])
            n_p +=1
        elif ((imt[k1,0,kz,2]==1) & (imt[k1,1,kz,2]==0)):
            inp_p.append([imt[k1,0,kz,1],imt[k1,1,kz,1]])
            tar_p.append([1,0,imt[k1,0,kz,3],imt[k1,1,kz,3],imt[k1,0,kz,0],imt[k1,1,kz,0],imt[k1,0,kz,4],imt[k1,1,kz,4]])
            n_p +=1
        elif ((imt[k1,0,kz,2]==0) & (imt[k1,1,kz,2]==1)):
            inp_p.append([imt[k1,0,kz,1],imt[k1,1,kz,1]])
            tar_p.append([0,1,imt[k1,0,kz,3],imt[k1,1,kz,3],imt[k1,0,kz,0],imt[k1,1,kz,0],imt[k1,0,kz,4],imt[k1,1,kz,4]])
            n_p +=1          
        elif ((imt[k1,0,kz,2]==0) & (imt[k1,1,kz,2]==0)):
            inp_n.append([imt[k1,0,kz,1],imt[k1,1,kz,1]])
            tar_n.append([0,0,imt[k1,0,kz,3],imt[k1,1,kz,3],imt[k1,0,kz,0],imt[k1,1,kz,0],imt[k1,0,kz,4],imt[k1,1,kz,4]])
            n_n +=1 
        else:
            n_pp +=1

print(n_p, n_n, n_pp)

550410 1471652 89410


In [ ]:
# For the training purpose, create a balanced training set.

# Determine the smaller length
min_len = min(len(inp_p), len(inp_n))

# Create balanced training sets
inp_tr = np.array(inp_p[:min_len] + inp_n[:min_len])
tar_tr = np.array(tar_p[:min_len] + tar_n[:min_len])

In [30]:
# Save the training or testing set.

np.save('Data_sets/'+f'inp_ch{ch}', inp_tr)
np.save('Data_sets/'+f'tar_ch{ch}', tar_tr)